In [3]:
import pandas as pd
import numpy as np

url = "../data/data_podcasts_title_content.csv"
df = pd.read_csv(url)

In [4]:
df.head(10)

,titles,content
0,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
1,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
2,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
3,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
4,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
5,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
6,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
7,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
8,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...
9,COMO ATINGIR A ALTA PERFORMANCE (com Eslen Del...,Title: COMO ATINGIR A ALTA PERFORMANCE (com Es...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer() # Important note here: there are many default parameters that do not show in this call.
X = vectorizer.fit_transform(df['content'])

In [10]:
df['content'].values[100]

'Title: Entendendo o SONO | Os Sócios Podcast #77\nViews count: 201810\nPublish date: 2022-06-23 00:00:00\nAuthor: Podcast Completo\nDescription: Unknown\nLink video: https://www.youtube.com/watch?v=chYBWwliAWM\n\na concepção de Caio sonho é um aviso de que algo vai acontecer quando a pessoa manda mensagem preocupada nossa Bruno sonhei com você e com seu uma coisa ruim eu falei cara fica tranquilo que se tudo que vocês sonham com a gente fosse acontecer Não teremos agenda para transar com todo mundo essa parte e eu também já tá grávida do 20º fino exatamente então Fiquem tranquilos quanto a isso mas voltando saindo agora do sonho para parte do Sono novamente muita gente comenta do chamado ciclo circadiano na Ilha do latim de cerca de um dia sim ao longo da evolução de ficou alinhado com isso um Quando o Sol nasce a gente tá acordando E aí quando o sol está se pondo a gente já começa a ficar mais relaxado pronto para o período Leiria né da noite de seria o ideal Então como é que tá a su

In [11]:
query = "longo da evolução de ficou alinhado com isso um Quando o Sol nasce a gente tá acordando E aí quando o sol está se pondo a gente já começa a ficar mais relaxado pronto para o período Leiria né da noite de seria o ideal Então como é que tá a sua relação do ciclo circadiano hoje com as pessoas e como que isso afeta"
Q = vectorizer.transform([query])

In [12]:
R = X.dot(Q.T)

In [13]:
idxs_and_scores = sorted([[idx, x[0]]for idx, x in enumerate(R.toarray())], key=lambda x: x[1], reverse=True)

In [21]:
idxs = np.array(idxs_and_scores[:10])[:, 0].astype(int)

In [22]:
df.iloc[idxs]

,titles,content
100,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
99,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
114,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
115,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
222,Puravida: Vitamina D e Pés no Solo | Os Sócios...,Title: Puravida: Vitamina D e Pés no Solo | Os...
168,COMO FUNCIONA O CÉREBRO | Os Sócios Podcast #43,Title: COMO FUNCIONA O CÉREBRO | Os Sócios Pod...
108,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
106,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
221,Puravida: Vitamina D e Pés no Solo | Os Sócios...,Title: Puravida: Vitamina D e Pés no Solo | Os...
104,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...


In [23]:
class Retriever:
    def __init__(self, path):
        self.df = pd.read_csv(path)
        self.vectorizer = TfidfVectorizer() # Important note here: there are many default parameters that do not show in this call.
        self.X = vectorizer.fit_transform(self.df['content'])
    
    def invoke(self, query, k=3):
        Q = vectorizer.transform([query])
        R = X.dot(Q.T)
        idxs_and_scores = sorted([[idx, x[0]]for idx, x in enumerate(R.toarray())], key=lambda x: x[1], reverse=True)
        retrieved = np.array(idxs_and_scores[:k])
        idxs = np.array(retrieved)[:, 0].astype(int)
        return df.iloc[idxs]

In [24]:
retriever = Retriever(path="../data/data_podcasts_title_content.csv")

In [26]:
query = "longo da evolução de ficou alinhado com isso um Quando o Sol nasce a gente tá acordando E aí quando o sol está se pondo a gente já começa a ficar mais relaxado pronto para o período Leiria né da noite de seria o ideal Então como é que tá a sua relação do ciclo circadiano hoje com as pessoas e como que isso afeta"

retriever.invoke(query, k=5)

,titles,content
100,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
99,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
114,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
115,Entendendo o SONO | Os Sócios Podcast #77,Title: Entendendo o SONO | Os Sócios Podcast #...
222,Puravida: Vitamina D e Pés no Solo | Os Sócios...,Title: Puravida: Vitamina D e Pés no Solo | Os...
